# Задание

Обучить decoder-encoder модель предсказывать ответ системы на запрос пользователя.

1. Проверить работу модели в сценарии близком к выбранному датасету в zero-shot режиме.
2. Посчитать метрики bleu и meteor исходной модели на валидационном сабсете выбранного датасета.
3. Дообучить модель на обучающем сабсете выбранного датасета.
4. Проверить работу модели в сценарии близком к выбранному датасету.
5. Посчитать метрики bleu и meteor обученной модели на валидационном сабсете выбранного датасета.

# Инициализация

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('cointegrated/rut5-small-chitchat2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/640k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
model = T5ForConditionalGeneration.from_pretrained('cointegrated/rut5-small-chitchat2')

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/259M [00:00<?, ?B/s]

# Тестовый запуск

In [ ]:
text = 'Привет! Расскажи, как твои дела?'

In [ ]:
inputs = tokenizer(text, return_tensors='pt')

In [ ]:
with torch.no_grad():
    hypotheses = model.generate(
        **inputs,
        temperature=0.9,
        do_sample=True,
        # sampling or greedy decoding
        top_p=0.7,
        num_return_sequences=3,
        repetition_penalty=2.5,
        max_length=32,
    )

In [ ]:
for h in hypotheses:
    print(tokenizer.decode(h, skip_special_tokens=True))

Что это такое?
У меня всё хорошо, что ты делаешь?
А где же это?


# Fine-Tuning

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df = load_dataset("zjkarina/matreshka")
train = df['train'].to_pandas()
test, valid = train_test_split(df['validation'].to_pandas(), test_size=0.5)

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-287def2e1da553(…):   0%|          | 0.00/3.28M [00:00<?, ?B/s]

data/validation-00000-of-00001-831bf3d82(…):   0%|          | 0.00/811k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6655 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1664 [00:00<?, ? examples/s]

**Вспомогательные функции**

Преобразования данных требуемые для подачи и использования в T5 образных моделях

In [ ]:
def _df_to_t5_format(df):
    user_bot = []
    for id, row in df.iterrows():
        row_role = row['role']
        row_dialog = row['dialog']
        if row_role is None or row_dialog is None:
            continue

        assert len(row_role) == len(row_dialog)

        row_role = [r.lower() for r in row_role]
        if any(r not in ['user', 'bot'] for r in row_role):
            continue

        role, dialog = _shrink_roles_and_dialogue(row_role, row_dialog)

        if len(role) <= 1:
            continue

        assert role[0] == 'user'
        assert role[-1] == 'bot'

        i = 0
        while i < len(role):
            user_bot.append((dialog[i], dialog[i + 1]))
            i += 2

    return user_bot

In [ ]:
def _shrink_roles_and_dialogue(row_role, row_dialog):
    shrinked_role = []  # [user, bot, bot, user] -> [user, bot, user]
    shrinked_dialog = [] # [a, b, c, d, e] -> [ab, cd, e]
    prev_role = ''
    for i in range(len(row_dialog)):
        if row_role[i] != prev_role:
            shrinked_role.append(row_role[i])
            shrinked_dialog.append(row_dialog[i])
        else:
            shrinked_dialog[-1] += ' ' + row_dialog[i]
        prev_role = row_role[i]

    assert len(shrinked_dialog) == len(shrinked_role)

    shrinked_role, shrinked_dialog = _drop_first_bot_last_user_utterings(shrinked_role, shrinked_dialog)

    return shrinked_role, shrinked_dialog

In [ ]:
def _drop_first_bot_last_user_utterings(role, dialog):
    if role[-1] == 'user':
        role, dialog = role[:-1], dialog[:-1]
    if role[0] == 'bot':
        role, dialog = role[1:], dialog[1:]
    return role, dialog

In [ ]:
t5train = _df_to_t5_format(train)
t5valid = _df_to_t5_format(valid)
t5test = _df_to_t5_format(test)

df_train = pd.DataFrame(t5train, columns=['source_text', 'target_text']) # для объединения фразы пользователя и фразы модели
df_val = pd.DataFrame(t5valid, columns=['source_text', 'target_text'])
df_test = pd.DataFrame(t5test, columns=['source_text', 'target_text'])

ANSWER_PREFIX = 'ответь: ' # исп. префикса
df_train['source_text'] = ANSWER_PREFIX + df_train['source_text'] # prefix + текст, на который модель должна ответить
df_val['source_text'] = ANSWER_PREFIX + df_val['source_text']
df_test['source_text'] = ANSWER_PREFIX + df_test['source_text']

Токенизация для T5

In [ ]:
def tokenize_data(data, column_name, max_length=64):
    source_texts = data[column_name].tolist()
    tokens = tokenizer.batch_encode_plus( # batch_encode — для кодирования пар текстов
        source_texts,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    return tokens

In [ ]:
train_tokens = tokenize_data(df_train, column_name='source_text') # encoder
train_target_tokens = tokenize_data(df_train, column_name='target_text') # decoder

val_tokens = tokenize_data(df_val, column_name='source_text')
val_target_tokens = tokenize_data(df_val, column_name='target_text')

test_tokens = tokenize_data(df_test, column_name='source_text')
test_target_tokens = tokenize_data(df_test, column_name='target_text')

In [ ]:
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(
    train_tokens['input_ids'],
    train_tokens['attention_mask'],
    train_target_tokens['input_ids'],
    train_target_tokens['attention_mask'],
)

val_dataset = TensorDataset(
    val_tokens['input_ids'],
    val_tokens['attention_mask'],
    val_target_tokens['input_ids'],
    val_target_tokens['attention_mask'],
)

test_dataset = TensorDataset(
    test_tokens['input_ids'],
    test_tokens['attention_mask'],
    test_target_tokens['input_ids'],
    test_target_tokens['attention_mask'],
)

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [ ]:
device = 'mps' if torch.backends.mps.is_built() else 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

cuda


In [ ]:
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(20100, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(20100, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

**Функции обучения**

In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm

def train_epoch(
    model,
    loader: DataLoader,
    epoch,
    num_epochs,
    optimizer,
    mode,
):
    total_loss = 0

    for x_input_ids, x_attention_mask, y_input_ids, y_attention_mask in tqdm(loader,
        desc=f'{mode} epoch {epoch}/{num_epochs}...'):

        if mode == 'Training':
            optimizer.zero_grad()

        x_input_ids = x_input_ids.to(device)
        x_attention_mask = x_attention_mask.to(device)
        y_input_ids = y_input_ids.to(device)
        y_attention_mask = y_attention_mask.to(device)

        outputs = model(
            input_ids=x_input_ids,
            attention_mask=x_attention_mask,
            labels=y_input_ids,
            decoder_attention_mask=y_attention_mask,
            return_dict=True,
        )

        outputs_loss = outputs.loss
        total_loss += outputs_loss.item()

        if mode == 'Training':
            outputs_loss.backward()
            optimizer.step()

    loss = total_loss / len(loader)
    print(f'{mode} epoch {epoch + 1}/{num_epochs}: {mode} Loss: {loss:.4f}')

In [ ]:
def train(
    model,
    train_loader,
    val_loader,
    optimizer,
    num_epochs=5,
):
    for epoch in range(num_epochs):
        model.train()
        train_epoch(model, train_loader, epoch, num_epochs, optimizer, mode='Training')

        model.eval()
        with torch.no_grad():
            train_epoch(model, val_loader, epoch, num_epochs, optimizer, mode='Validating')

**Обучение**

In [ ]:
train(model, train_loader, val_loader, optimizer, num_epochs=5)

Training epoch 0/5...: 100%|██████████| 1858/1858 [03:14<00:00,  9.53it/s]


Training epoch 1/5: Training Loss: 3.3768


Validating epoch 0/5...: 100%|██████████| 232/232 [00:07<00:00, 31.72it/s]


Validating epoch 1/5: Validating Loss: 1.0355


Training epoch 1/5...: 100%|██████████| 1858/1858 [03:15<00:00,  9.51it/s]


...

Validating epoch 4/5: Validating Loss: 0.8926


Training epoch 4/5...: 100%|██████████| 1858/1858 [03:14<00:00,  9.54it/s]


Training epoch 5/5: Training Loss: 1.0198


Validating epoch 4/5...: 100%|██████████| 232/232 [00:06<00:00, 34.30it/s]

Validating epoch 5/5: Validating Loss: 0.8745


# Получение ответов FT модели

In [ ]:
inputs['input_ids'] = inputs['input_ids'].to(device)
inputs['attention_mask'] = inputs['attention_mask'].to(device)

In [ ]:
with torch.no_grad():
    # https://huggingface.co/docs/transformers/main/en/main_classes/text_generation
    hypotheses = model.generate(
        **inputs,
        temperature=0.9,
        do_sample=True,             # sampling or greedy decoding
        # (at each decoding step selects the token with the highest prob without considering the impact on future tokens)
        top_p=0.7,
        num_return_sequences=3,
        repetition_penalty=2.5,     # https://arxiv.org/pdf/1909.05858.pdf
        max_length=32,
    )

In [ ]:
for h in hypotheses:
    print(tokenizer.decode(h, skip_special_tokens=True))

Привет! У меня все хорошо, как у тебя?
Привет! У меня все хорошо, но у тебя тоже отлично. А ты?
Привет! У меня все хорошо, что у тебя?


# Вычисление BLEU-метрик на валидационных данных

In [ ]:
def generate(
    model,
    inputs,
    num_return_sequences=5,
):
    return model.generate(
        **inputs,
        temperature=0.9,
        do_sample=True,
        top_p=0.7,
        num_return_sequences=num_return_sequences,
        repetition_penalty=2.5,
        max_length=64,
    )

In [ ]:
!pip install torchmetrics;

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [ ]:
import re
from torchmetrics.text import BLEUScore

In [ ]:
from torchmetrics.text import BLEUScore
import re
from tqdm import tqdm
import torch
from transformers import T5ForConditionalGeneration

bleu = BLEUScore(n_gram=1)
cnt = 100

initial_model = T5ForConditionalGeneration.from_pretrained("cointegrated/rut5-small-chitchat2").to(device)
finetuned_model = model

initial_bleu_score = 0.
finetuned_bleu_score = 0.

with torch.no_grad():
    for _, row in tqdm(df_val.head(cnt).iterrows(), total=cnt):
        source = row['source_text']
        target = row['target_text']

        inputs_init = tokenizer(source, return_tensors="pt")
        for key in inputs_init.keys():
            inputs_init[key] = inputs_init[key].to(device)

        hypotheses_init = generate(initial_model, inputs_init, num_return_sequences=1)
        pred_init = tokenizer.decode(hypotheses_init[0], skip_special_tokens=True)

        inputs_fine = tokenizer(source, return_tensors="pt")
        for key in inputs_fine.keys():
            inputs_fine[key] = inputs_fine[key].to(device)

        hypotheses_fine = generate(finetuned_model, inputs_fine, num_return_sequences=1)
        pred_fine = tokenizer.decode(hypotheses_fine[0], skip_special_tokens=True)

        # Обрезаем до первого предложения
        shrincked_target = re.split(r'\.|!|\?', target)[0]
        shrincked_init = re.split(r'\.|!|\?', pred_init)[0]
        shrincked_fine = re.split(r'\.|!|\?', pred_fine)[0]

        initial_bleu_score += bleu(shrincked_init, shrincked_target).item()
        finetuned_bleu_score += bleu(shrincked_fine, shrincked_target).item()

print('avg bleu:', finetuned_bleu_score * 1. / cnt)
print('avg initial bleu:', initial_bleu_score * 1. / cnt)

100%|██████████| 100/100 [00:41<00:00,  2.40it/s]

avg bleu: 0.1471860490553081
avg initial bleu: 0.09354216763749719


# Вычисление METEOR-метрик на валидационных данных

In [ ]:
!pip install evaluate;

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.8 MB/s eta 0:00:00


In [ ]:
import evaluate

meteor = evaluate.load('meteor')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
avg_meteor_score = 0.
initial_meteor_score = 0.
cnt = 100

initial_model = T5ForConditionalGeneration.from_pretrained('cointegrated/rut5-small-chitchat2')
initial_model = initial_model.to(device)

with torch.no_grad():
    for _, row in tqdm(df_val.head(cnt).iterrows(), total=cnt):
        source = row['source_text']
        target = row['target_text']

        inputs = tokenizer(source, return_tensors='pt')
        for key in inputs.keys():
            inputs[key] = inputs[key].to(device)

        hypotheses = generate(model, inputs, num_return_sequences=1)
        not_fine_tuned_hypotheses = generate(initial_model, inputs, num_return_sequences=1)

        pred = tokenizer.decode(hypotheses[0], skip_special_tokens=True)
        not_fine_tuned_preds = tokenizer.decode(not_fine_tuned_hypotheses[0], skip_special_tokens=True)

        shrinked_target = re.split('\.|\?', target)[0]
        shrinked_pred = re.split('\.|\?', pred)[0]
        shrinked_initial_preds = re.split('\.|\?', not_fine_tuned_preds)[0]

        metric = meteor.compute(predictions=[shrinked_pred], references=[shrinked_target])
        initial_metric = meteor.compute(predictions=[shrinked_initial_preds], references=[shrinked_target])

        avg_meteor_score += metric['meteor']
        initial_meteor_score += initial_metric['meteor']

print('avg meteor:', avg_meteor_score * 1. / cnt)
print('avg initial meteor:', initial_meteor_score * 1. / cnt)

100%|██████████| 100/100 [00:42<00:00,  2.35it/s]

avg meteor: 0.15262721194634596
avg initial meteor: 0.04534620526024668


# Выводы

1. Обе метрики подросли
- METEOR вырос относительно сильно
2. Метрики очень низкого уровня
- Однако у нас не так много данных для оценки: один эталон и короткие ответы
3. Ответ после дообучения действительно заметно лучше, чем "из коробки".
- Ответ лаконичен, с небольшими галюцинациями